# Data Exercise #4

In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import scipy.stats
import math
import time
import datetime

Please apply the following filters to your data: (i) Trades and quotes must be between [09:30,16:00].
Remove both sides of the quote if (ii) the bid or ask price is zero, (ii) the bid or ask quantity is
zero; (iii) the bid is greater than the ask, (iv) if the percentage spread is greater than 10%

In [21]:
def time_nano(time_float):
    ihour = np.int64((time_float/10000))
    imin = ((time_float-ihour*10000)/100).astype(np.int64)
    fsec = (time_float-ihour*10000-imin*100).astype(np.float64)
    tsec = ihour*3600+imin*60+fsec
    return tsec

In [39]:
data_file = 'CWS_quotes_2021_08_24.csv.gz'
qd = pd.read_csv(data_file)
qd['ftime']=quote_data['TIME_M'].apply(lambda x: float(str(x.replace(':',''))))
qd['quote_tsec']=quote_data['ftime'].apply(lambda x: time_nano(x))
qd=qd.drop(columns=['SYM_ROOT','SYM_SUFFIX','LULD_BBO_Indicator','SIP_MESSAGE_ID','RPI','SSR','NATL_BBO_LULD','FINRA_ADF_MPID','FINRA_BBO_IND','FINRA_ADF_TIME','QU_CANCEL','QU_SOURCE','PART_TIME', 'FINRA_ADF_MQUO_IND','SECSTAT_IND'])
qd.head()

C:\Users\jessi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
0,20210824,3:59:00.037252096,Z,0.00,0,0.0,0,R,6676,O,35900.037252,14340.037252
1,20210824,3:59:00.174957056,K,0.00,0,0.0,0,R,7351,O,35900.174957,14340.174957
2,20210824,6:58:00.178641920,Y,0.00,0,0.0,0,R,138341,O,65800.178642,25080.178642
3,20210824,7:00:00.032725248,K,11.93,8,0.0,0,R,140578,G,70000.032725,25200.032725
4,20210824,7:10:48.309867520,L,0.00,0,0.0,0,R,161866,A,71048.309868,25848.309868


In [41]:
early = qd[ qd['ftime'] <= 	93000 ].index
qd.drop(early , inplace=True)
late = qd[ qd['ftime'] > 	160000 ].index
qd.drop(late , inplace=True)
qd.head()

,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
121,20210824,9:30:00.222236928,P,48.18,100,49.26,100,R,1076009,A,93000.222237,34200.222237
122,20210824,9:30:00.222777600,P,48.71,1,49.26,100,R,1076022,G,93000.222778,34200.222778
123,20210824,9:30:00.222781952,P,48.18,100,49.26,100,R,1076023,G,93000.222782,34200.222782
124,20210824,9:30:00.222942208,K,45.34,1,52.17,1,R,1076024,A,93000.222942,34200.222942
125,20210824,9:30:00.223729152,H,0.00,0,0.00,0,R,1076047,A,93000.223729,34200.223729


In [43]:
bidzero = qd[ qd['BID'] == 0 ].index
qd.drop(bidzero , inplace=True)
bidSIZzero = qd[ qd['BIDSIZ'] == 0 ].index
qd.drop(bidSIZzero , inplace=True)
qd.head()

,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
121,20210824,9:30:00.222236928,P,48.18,100,49.26,100,R,1076009,A,93000.222237,34200.222237
122,20210824,9:30:00.222777600,P,48.71,1,49.26,100,R,1076022,G,93000.222778,34200.222778
123,20210824,9:30:00.222781952,P,48.18,100,49.26,100,R,1076023,G,93000.222782,34200.222782
124,20210824,9:30:00.222942208,K,45.34,1,52.17,1,R,1076024,A,93000.222942,34200.222942
126,20210824,9:30:00.344557056,P,48.12,16,49.26,100,R,1079205,U,93000.344557,34200.344557


In [45]:
ASKzero = qd[ qd['ASK'] == 0 ].index
qd.drop(ASKzero , inplace=True)
ASKSIZzero = qd[ qd['ASKSIZ'] == 0 ].index
qd.drop(ASKSIZzero , inplace=True)
qd.head()

,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
121,20210824,9:30:00.222236928,P,48.18,100,49.26,100,R,1076009,A,93000.222237,34200.222237
122,20210824,9:30:00.222777600,P,48.71,1,49.26,100,R,1076022,G,93000.222778,34200.222778
123,20210824,9:30:00.222781952,P,48.18,100,49.26,100,R,1076023,G,93000.222782,34200.222782
124,20210824,9:30:00.222942208,K,45.34,1,52.17,1,R,1076024,A,93000.222942,34200.222942
126,20210824,9:30:00.344557056,P,48.12,16,49.26,100,R,1079205,U,93000.344557,34200.344557


In [47]:
bidlg = qd[ qd['BID'] > qd['ASK'] ].index
qd.drop(bidlg , inplace=True)
qd.head()

,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
121,20210824,9:30:00.222236928,P,48.18,100,49.26,100,R,1076009,A,93000.222237,34200.222237
122,20210824,9:30:00.222777600,P,48.71,1,49.26,100,R,1076022,G,93000.222778,34200.222778
123,20210824,9:30:00.222781952,P,48.18,100,49.26,100,R,1076023,G,93000.222782,34200.222782
124,20210824,9:30:00.222942208,K,45.34,1,52.17,1,R,1076024,A,93000.222942,34200.222942
126,20210824,9:30:00.344557056,P,48.12,16,49.26,100,R,1079205,U,93000.344557,34200.344557


In [49]:
spread = qd[ qd['BID']*1.1 < qd['ASK'] ].index
qd.drop(spread , inplace=True)
qd.head()

,DATE,TIME_M,EX,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,ftime,quote_tsec
121,20210824,9:30:00.222236928,P,48.18,100,49.26,100,R,1076009,A,93000.222237,34200.222237
122,20210824,9:30:00.222777600,P,48.71,1,49.26,100,R,1076022,G,93000.222778,34200.222778
123,20210824,9:30:00.222781952,P,48.18,100,49.26,100,R,1076023,G,93000.222782,34200.222782
126,20210824,9:30:00.344557056,P,48.12,16,49.26,100,R,1079205,U,93000.344557,34200.344557
127,20210824,9:30:00.345322752,P,48.12,16,49.17,100,R,1079214,U,93000.345323,34200.345323


In [50]:
len(qd)

44408